In [29]:
%pip install scipy


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
from scipy.optimize import fmin
from time import time

In [2]:
def generate_grid(start: float, end: float, interval: float):
    # python の標準パッケージのみで実装
    grid = []
    i = start
    while i <= end:
        grid.append(i)
        i += interval
    return grid

In [63]:
def calc_residual(
    w: float, theta0: float, theta1: float, r: float, beta: float, gamma: float
) -> float:
    a = theta0 + theta1 * w
    if a == 0.0:
        a += 1e-05
    # if w - a < 0:
    #     raise ValueError("w < a!!!")
    # if w - a == 0:
    #     a -= 1e-05
        # raise ValueError("w == a!!!")
    # residual = beta * (1.0 + r)*(((1.0+r)*a) / (w - a))**(- gamma) - 1.0
    residual = beta * (1.0 + r) * ((w - a) / ((1.0 + r) * a)) ** gamma - 1.0
    # residual = (beta*(1+r))*((((1+r)*a)**(-gamma))/((w-a)**(-gamma)))-1
    return residual



In [64]:
def gen_ssr_func(w_list, r, beta, gamma):
    def calc_ssr(theta) -> float:
        theta0, theta1 = theta
        ssr = 0.0
        for w in w_list:
            try:
                ssr += calc_residual(w, theta0, theta1, r, beta, gamma) ** 2
            except ValueError:  # as e:
                # print(e)
                continue
        return ssr

    return calc_ssr

In [65]:
def main():
    # カリブレーション
    beta = 0.985**30
    r = 1.025**30 - 1.0
    gamma: float = 2.0

    # 状態変数 w のグリッドを生成する
    w_list = generate_grid(0.1, 1.0, 0.025)

    # theta0, theta1 をインプットに残差2乗和を計算する関数を定義
    calc_ssr = gen_ssr_func(w_list, r, beta, gamma)

    # result = calc_rss([0.0, 0.35])

    try:
        result = fmin(calc_ssr, x0=[-0.5, -0.5], maxiter=2000, ftol=1e-10)
        return result
    except Exception:
        import traceback

        with open("error.log", "a") as f:
            traceback.print_exc(file=f)



In [66]:
# calc_ssrを3Dプロットする
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [67]:
# カリブレーション
beta = 0.985**30
r = 1.025**30 - 1.0
gamma: float = 2.0

# 状態変数 w のグリッドを生成する
w_list = generate_grid(0.1, 1.0, 0.025)

# theta0, theta1 をインプットに残差2乗和を計算する関数を定義
calc_ssr = gen_ssr_func(w_list, r, beta, gamma)

In [74]:
fig = plt.figure()
ax = fig.add_subplot(111, projection="3d")

# x と y の範囲の設定
theta0_range = np.linspace(-1.0, 1, 100)
theta1_range = np.linspace(-1.0, 1, 100)
theta0, theta1 = np.meshgrid(theta0_range, theta1_range)

# 関数を記述
rss = calc_ssr([theta0, theta1])

ax.plot_surface(theta0, theta1, rss, cmap=) # type: ignore
# ax.contour(theta0, theta1, rss, colors = "gray", offset = -1)  # 底面に等高線を描画

# 自動で軸を設定する場合は記述なし

# 手動で軸を設定する場合
# ax.set_xlim(-20, 20)
# ax.set_ylim(-20, 20)
# ax.set_zlim(0, 10000)

ax.set_xlabel("theta0")
ax.set_ylabel("theta1")
ax.set_zlabel("RSS")

plt.show() 

SyntaxError: expected argument value expression (2063706159.py, line 12)

In [49]:

t0 = time()
pf = main()
t1 = time()
print(f"Computed policy function = \n\n{pf}\n\nin {t1 - t0} seconds.")


Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 53
         Function evaluations: 100
Computed policy function = 

[ 5.29991785e-07 -1.22424492e+00]

in 0.017278194427490234 seconds.


In [ ]:
# 原因と対処法
# 目的関数が綺麗な形じゃなくて、最適化関数を通した結果、変なかいが出てきちゃう問題に対して、どうすれば良いか調べる。